# 🎨 anime-character-generator
## Stable Diffusion + PyTorch (Colab版・シンプル)

Google Colab 環境で動作確認済みの簡易版です。

In [ ]:
# Step 1: GPU 確認
import torch
print(f"✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Step 2: ライブラリインストール（最新版・競合回避）
!pip install -q --upgrade setuptools wheel
!pip install -q diffusers transformers accelerate safetensors
!pip install -q pillow torchvision
print("✅ Installation complete!")

In [ ]:
# Step 3: インポート
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os
import matplotlib.pyplot as plt

print("✅ All libraries imported!")

In [ ]:
# Step 4: モデルロード
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print(f"📦 Loading model... (device: {device})")

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype,
    safety_checker=None
)
pipe = pipe.to(device)
pipe.enable_attention_slicing()

print("✅ Model ready!")

In [ ]:
# Step 5: 感情バリエーション生成
import os
os.makedirs("outputs/emotions", exist_ok=True)

emotions = {
    "happy": "happy smile, cheerful, joyful",
    "angry": "angry expression, intense eyes",
    "sad": "sad expression, melancholic",
    "surprised": "surprised expression, wide eyes"
}

base = "1girl, anime character, masterpiece, high quality"
emotion_images = {}

print("\n🎭 GENERATING EMOTIONS...\n")

for emotion_name, emotion_desc in emotions.items():
    prompt = f"{base}, {emotion_desc}"
    print(f"  [{emotion_name.upper()}] Generating...", end="")
    
    with torch.no_grad():
        image = pipe(
            prompt=prompt,
            negative_prompt="low quality, blurry",
            num_inference_steps=20,
            guidance_scale=7.0,
            height=512,
            width=512
        ).images[0]
    
    filepath = f"outputs/emotions/character_{emotion_name}.png"
    image.save(filepath)
    emotion_images[emotion_name] = image
    print(" ✅")

print("\n✅ Emotions complete!")

In [ ]:
# Step 6: スタイルバリエーション生成
os.makedirs("outputs/styles", exist_ok=True)

styles = {
    "with_hat": "wearing hat",
    "with_earrings": "wearing earrings",
    "formal": "formal dress, elegant",
    "casual": "casual outfit",
    "with_makeup": "with makeup, beautiful",
    "glasses": "wearing glasses"
}

style_images = {}

print("👗 GENERATING STYLES...\n")

for style_name, style_desc in styles.items():
    prompt = f"{base}, {style_desc}"
    print(f"  [{style_name.upper()}] Generating...", end="")
    
    with torch.no_grad():
        image = pipe(
            prompt=prompt,
            negative_prompt="low quality, blurry",
            num_inference_steps=20,
            guidance_scale=7.0,
            height=512,
            width=512
        ).images[0]
    
    filepath = f"outputs/styles/character_{style_name}.png"
    image.save(filepath)
    style_images[style_name] = image
    print(" ✅")

print("\n✅ Styles complete!")

In [ ]:
# Step 7: 結果表示（感情）
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, (name, img) in enumerate(emotion_images.items()):
    axes[idx].imshow(img)
    axes[idx].set_title(name.upper(), fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig("emotion_results.png", dpi=100, bbox_inches='tight')
plt.show()
print("✅ Emotion results saved!")

In [ ]:
# Step 8: 結果表示（スタイル）
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (name, img) in enumerate(style_images.items()):
    axes[idx].imshow(img)
    axes[idx].set_title(name.upper(), fontsize=11, fontweight='bold')
    axes[idx].axis('off')

fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig("style_results.png", dpi=100, bbox_inches='tight')
plt.show()
print("✅ Style results saved!")

In [ ]:
# Step 9: ダウンロード準備
print("\n" + "="*60)
print("✅ GENERATION COMPLETE!")
print("="*60)
print(f"\n📁 Generated {len(emotion_images) + len(style_images)} images")
print("\n📥 To download:")
print("   1. Left sidebar → Files icon")
print("   2. outputs/ folder → Right click")
print("   3. Download")
print("\n✨ Success!")